# I. Xử lý dữ liệu
## Trích xuất đặc trưng của file PE. Bao gồm 70 đặc trưng:

IMAGE_DOS_HEADER (6)
- e_cblp
- e_cp
- e_cparhdr
- e_maxalloc
- e_sp
- e_lfanew

FILE_HEADER (17)
- NumberOfSections
- CreationYear
- FH_char0
- FH_char1
- FH_char2
- FH_char3
- FH_char4
- FH_char5
- FH_char6
- FH_char7
- FH_char8
- FH_char9
- FH_char10
- FH_char11
- FH_char12
- FH_char13
- FH_char14

OPTIONAL_HEADER (37)
- MajorLinkerVersion
- MinorLinkerVersion
- SizeOfCode
- SizeOfInitializedData
- SizeOfUninitializedData
- AddressOfEntryPoint
- BaseOfCode
- BaseOfData
- ImageBase
- SectionAlignment
- FileAlignment
- MajorOperatingSystemVersion
- MinorOperatingSystemVersion
- MajorImageVersion
- MinorImageVersion
- MajorSubsystemVersion
- MinorSubsystemVersion
- SizeOfImage
- SizeOfHeaders
- CheckSum
- Subsystem
- OH_DLLchar0
- OH_DLLchar1
- OH_DLLchar2
- OH_DLLchar3
- OH_DLLchar4
- OH_DLLchar5
- OH_DLLchar6
- OH_DLLchar7
- OH_DLLchar8
- OH_DLLchar9
- OH_DLLchar10
- SizeOfStackReserve
- SizeOfStackCommit
- SizeOfHeapReserve
- SizeOfHeapCommit
- LoaderFlags

Derived Header (8)
- sus_sections
- non_sus_sections
- packer
- packer_type
- E_text
- E_data
- filesize
- E_file
- fileinfo

In [1]:
# class pe_features():

#     IMAGE_DOS_HEADER = [
#                         "e_cblp",
#                         "e_cp", 
#                         "e_cparhdr",
#                         "e_maxalloc",
#                         "e_sp",
#                         "e_lfanew"]

#     FILE_HEADER= ["NumberOfSections","CreationYear"] + [ "FH_char" + str(i) for i in range(15)]
                

#     OPTIONAL_HEADER1 = [
#                         "MajorLinkerVersion",
#                         "MinorLinkerVersion",
#                         "SizeOfCode",
#                         "SizeOfInitializedData",
#                         "SizeOfUninitializedData",
#                         "AddressOfEntryPoint",
#                         "BaseOfCode",
#                         "BaseOfData",
#                         "ImageBase",
#                         "SectionAlignment",
#                         "FileAlignment",
#                         "MajorOperatingSystemVersion",
#                         "MinorOperatingSystemVersion",
#                         "MajorImageVersion",
#                         "MinorImageVersion",
#                         "MajorSubsystemVersion",
#                         "MinorSubsystemVersion",
#                         "SizeOfImage",
#                         "SizeOfHeaders",
#                         "CheckSum",
#                         "Subsystem"] 
#     OPTIONAL_HEADER_DLL_char = [ "OH_DLLchar" + str(i) for i in range(11)]                   
                            
#     OPTIONAL_HEADER2 = [
#                         "SizeOfStackReserve",
#                         "SizeOfStackCommit",
#                         "SizeOfHeapReserve",
#                         "SizeOfHeapCommit",
#                         "LoaderFlags"]  # boolean check for zero or not
    
#     OPTIONAL_HEADER = OPTIONAL_HEADER1 + OPTIONAL_HEADER_DLL_char + OPTIONAL_HEADER2
#     Derived_header = ["sus_sections","non_sus_sections", "packer","E_text","E_data","filesize","E_file","fileinfo"]
#     # Derived_header = ["sus_sections","non_sus_sections", "packer","packer_type","E_text","E_data","filesize","E_file","fileinfo"]
    
#     def __init__(self, source, output, label):
#         self.source = source
#         self.output = output
#         self.type = label
#         self.rules= yara.compile('peid.yar')
        
#     def file_creation_year(self,seconds):
#         tmp = 1970 + ((int(seconds) / 86400) / 365)
#         return int(tmp in range (1980,2016)) 


#     def FILE_HEADER_Char_boolean_set(self,pe):
#         tmp = [pe.FILE_HEADER.IMAGE_FILE_RELOCS_STRIPPED,
#             pe.FILE_HEADER.IMAGE_FILE_EXECUTABLE_IMAGE,
#             pe.FILE_HEADER.IMAGE_FILE_LINE_NUMS_STRIPPED,
#             pe.FILE_HEADER.IMAGE_FILE_LOCAL_SYMS_STRIPPED,
#             pe.FILE_HEADER.IMAGE_FILE_AGGRESIVE_WS_TRIM,
#             pe.FILE_HEADER.IMAGE_FILE_LARGE_ADDRESS_AWARE,
#             pe.FILE_HEADER.IMAGE_FILE_BYTES_REVERSED_LO,
#             pe.FILE_HEADER.IMAGE_FILE_32BIT_MACHINE,
#             pe.FILE_HEADER.IMAGE_FILE_DEBUG_STRIPPED,
#             pe.FILE_HEADER.IMAGE_FILE_REMOVABLE_RUN_FROM_SWAP,
#             pe.FILE_HEADER.IMAGE_FILE_NET_RUN_FROM_SWAP,
#             pe.FILE_HEADER.IMAGE_FILE_SYSTEM,
#             pe.FILE_HEADER.IMAGE_FILE_DLL,
#             pe.FILE_HEADER.IMAGE_FILE_UP_SYSTEM_ONLY,
#             pe.FILE_HEADER.IMAGE_FILE_BYTES_REVERSED_HI
#             ]
#         return [int(s) for s in tmp]

#     def OPTIONAL_HEADER_DLLChar(self,pe):
#         tmp = [
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_DYNAMIC_BASE,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_FORCE_INTEGRITY,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NX_COMPAT,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_ISOLATION,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_SEH,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_NO_BIND,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_WDM_DRIVER,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_TERMINAL_SERVER_AWARE,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_HIGH_ENTROPY_VA,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_APPCONTAINER,
#             pe.OPTIONAL_HEADER.IMAGE_DLLCHARACTERISTICS_GUARD_CF
#             ]
#         return [int(s) for s in tmp]
#     def Optional_header_ImageBase(self,ImageBase):
#         result= 0
#         if ImageBase % (64 * 1024) == 0 and ImageBase in [268435456,65536,4194304]:
#             result = 1
#         return result
#     def Optional_header_SectionAlignment(self,SectionAlignment,FileAlignment):
#         """This is boolean function and will return 0 or 1 based on condidtions
#         that it SectionAlignment must be greater than or equal to FileAlignment
#         """
#         return int(SectionAlignment >= FileAlignment)
#     def Optional_header_FileAlignment(self,SectionAlignment,FileAlignment):
#         result =0
#         if SectionAlignment >= 512:
#             if FileAlignment % 2 == 0 and FileAlignment in range(512,65537):
#                 result =1
#         else: 
#             if FileAlignment == SectionAlignment:
#                 result = 1
#         return result

#     def Optional_header_SizeOfImage(self,SizeOfImage,SectionAlignment):
#         return int(SizeOfImage % SectionAlignment == 0)

#     def Optional_header_SizeOfHeaders(self,SizeOfHeaders,FileAlignment):
#         return int(SizeOfHeaders % FileAlignment == 0 )


#     def extract_dos_header(self,pe):
#         IMAGE_DOS_HEADER_data = [ 0 for i in range(6)]
#         try:
#             IMAGE_DOS_HEADER_data = [
#                                 pe.DOS_HEADER.e_cblp,
#                                 pe.DOS_HEADER.e_cp, 
#                                 pe.DOS_HEADER.e_cparhdr,
#                                 pe.DOS_HEADER.e_maxalloc,
#                                 pe.DOS_HEADER.e_sp,
#                                 pe.DOS_HEADER.e_lfanew]
#         except Exception as e:
#             # print(e)
#             pass
#         return IMAGE_DOS_HEADER_data

#     def extract_file_header(self,pe):	
#         FILE_HEADER_data = [ 0 for i in range(3)]
#         FILE_HEADER_char =  []
#         try:
#             FILE_HEADER_data = [ 
#                     pe.FILE_HEADER.NumberOfSections, 
#                     self.file_creation_year(pe.FILE_HEADER.TimeDateStamp)]
#             FILE_HEADER_char = self.FILE_HEADER_Char_boolean_set(pe)
#         except Exception as e:
#             # print(e)
#             pass
#         return FILE_HEADER_data + FILE_HEADER_char

#     def extract_optional_header(self,pe):
#         OPTIONAL_HEADER_data = [ 0 for i in range(21)]
#         DLL_char =[]
#         OPTIONAL_HEADER_data2 = [ 0 for i in range(6)]

#         try:
#             OPTIONAL_HEADER_data = [
#                 pe.OPTIONAL_HEADER.MajorLinkerVersion,
#                 pe.OPTIONAL_HEADER.MinorLinkerVersion,
#                 pe.OPTIONAL_HEADER.SizeOfCode,
#                 pe.OPTIONAL_HEADER.SizeOfInitializedData,
#                 pe.OPTIONAL_HEADER.SizeOfUninitializedData,
#                 pe.OPTIONAL_HEADER.AddressOfEntryPoint,
#                 pe.OPTIONAL_HEADER.BaseOfCode,
#                 pe.OPTIONAL_HEADER.BaseOfData,
#                 #Check the ImageBase for the condition
#                 self.Optional_header_ImageBase(pe.OPTIONAL_HEADER.ImageBase),
#                 # Checking for SectionAlignment condition
#                 self.Optional_header_SectionAlignment(pe.OPTIONAL_HEADER.SectionAlignment,pe.OPTIONAL_HEADER.FileAlignment),
#                 #Checking for FileAlignment condition
#                 self.Optional_header_FileAlignment(pe.OPTIONAL_HEADER.SectionAlignment,pe.OPTIONAL_HEADER.FileAlignment),
#                 pe.OPTIONAL_HEADER.MajorOperatingSystemVersion,
#                 pe.OPTIONAL_HEADER.MinorOperatingSystemVersion,
#                 pe.OPTIONAL_HEADER.MajorImageVersion,
#                 pe.OPTIONAL_HEADER.MinorImageVersion,
#                 pe.OPTIONAL_HEADER.MajorSubsystemVersion,
#                 pe.OPTIONAL_HEADER.MinorSubsystemVersion,
#                 #Checking size of Image
#                 self.Optional_header_SizeOfImage(pe.OPTIONAL_HEADER.SizeOfImage,pe.OPTIONAL_HEADER.SectionAlignment),
#                 #Checking for size of headers
#                 self.Optional_header_SizeOfHeaders(pe.OPTIONAL_HEADER.SizeOfHeaders,pe.OPTIONAL_HEADER.FileAlignment),
#                 pe.OPTIONAL_HEADER.CheckSum,
#                 pe.OPTIONAL_HEADER.Subsystem]

#             DLL_char = self.OPTIONAL_HEADER_DLLChar(pe)

#             OPTIONAL_HEADER_data2= [                
#                 pe.OPTIONAL_HEADER.SizeOfStackReserve,
#                 pe.OPTIONAL_HEADER.SizeOfStackCommit,
#                 pe.OPTIONAL_HEADER.SizeOfHeapReserve,
#                 pe.OPTIONAL_HEADER.SizeOfHeapCommit,
#                 int(pe.OPTIONAL_HEADER.LoaderFlags == 0) ]
#         except Exception as e:
#             # print(e)
#             pass
#         return OPTIONAL_HEADER_data + DLL_char + OPTIONAL_HEADER_data2

#     def get_count_suspicious_sections(self,pe):
#         result=[]
#         tmp =[]
#         benign_sections = set(['.text','.data','.rdata','.idata','.edata','.rsrc','.bss','.crt','.tls'])
#         for section in pe.sections:
#             tmp.append(section.Name.split(b'x00')[0])
#         non_sus_sections = len(set(tmp).intersection(benign_sections))
#         result=[len(tmp) - non_sus_sections, non_sus_sections]
#         return result


#     def check_packer(self,filepath):
#         result=[]
#         matches = self.rules.match(filepath)
#         if matches == []:
#             result.append([0,"NoPacker"])
#         else:
#             #result.append([1,matches['main'][0]['rule']])
#             result.append([1,matches[0]])
#         return result

#     def get_text_data_entropy(self,pe):
#         result=[0.0,0.0]
#         for section in pe.sections:
#             s_name = section.Name.split(b'x00')[0]
#             if s_name == ".text":
#                 result[0]= section.get_entropy()
#             elif s_name == ".data":
#                 result[1]= section.get_entropy()
#             else:
#                 pass
#         return result  

#     def get_file_bytes_size(self,filepath):
#         f = open(filepath, "rb")
#         data = f.read()
#         # byteArr = [ord(c) for c in f.read()]
#         #f.close()
#         fileSize = len(data)
#         return data, fileSize

#     def cal_byteFrequency(self,byteArr,fileSize):
#         freqList = []
#         for b in range(256):
#             ctr = 0
#             for byte in byteArr:
#                 if byte == b:
#                     ctr += 1
#             freqList.append(float(ctr) / fileSize)
#         return freqList

#     def get_file_entropy(self,filepath):
#         byteArr, fileSize = self.get_file_bytes_size(filepath)
#         freqList = self.cal_byteFrequency(byteArr,fileSize)
#         # Shannon entropy
#         ent = 0.0
#         for freq in freqList:
#             if freq > 0:
#                 ent +=  - freq * math.log(freq, 2)

#             #ent = -ent
#         return [fileSize,ent]
    
#     def get_fileinfo(self,pe):
#         result=[]
#         try:
#             FileVersion    = pe.FileInfo[0].StringTable[0].entries['FileVersion']
#             ProductVersion = pe.FileInfo[0].StringTable[0].entries['ProductVersion']
#             ProductName =    pe.FileInfo[0].StringTable[0].entries['ProductName']
#             CompanyName = pe.FileInfo[0].StringTable[0].entries['CompanyName']
#     #getting Lower and 
#             FileVersionLS    = pe.VS_FIXEDFILEINFO.FileVersionLS
#             FileVersionMS    = pe.VS_FIXEDFILEINFO.FileVersionMS
#             ProductVersionLS = pe.VS_FIXEDFILEINFO.ProductVersionLS
#             ProductVersionMS = pe.VS_FIXEDFILEINFO.ProductVersionMS
#         except Exception as e:
#             result=["error"]
#         #print "{} while opening {}".format(e,filepath)
#         else:
#     #shifting byte
#             FileVersion = (FileVersionMS >> 16, FileVersionMS & 0xFFFF, FileVersionLS >> 16, FileVersionLS & 0xFFFF)
#             ProductVersion = (ProductVersionMS >> 16, ProductVersionMS & 0xFFFF, ProductVersionLS >> 16, ProductVersionLS & 0xFFFF)
#             result = [FileVersion,ProductVersion,ProductName,CompanyName]
#         return int ( result[0] != 'error')

#     def write_csv_header(self):
#         if os.path.exists(self.output): return
#         filepath = self.output
#         header= self.IMAGE_DOS_HEADER + self.FILE_HEADER + self.OPTIONAL_HEADER + self.Derived_header
#         header.append("class")
#         csv_file= open(filepath,"a")
#         writer = csv.writer(csv_file, delimiter=',')
#         writer.writerow(header)
#         csv_file.close()

#     def extract_all(self,filepath):
#         data =[]


#         os.system(f'cp -f {filepath} /tmp/1.exe')
#         filepath = '/tmp/1.exe'

#         try:
#             pe = pefile.PE(filepath)
#         except Exception as e:
#             print(e)
#             pass
            
#         try:
#             data += self.extract_dos_header(pe)
#         except Exception as err:
#             print('ERROR EXTRACT DOS HEADER: ' + str(err))
#             return None

#         try:
#             data += self.extract_file_header(pe)
#         except Exception as err:
#             print('ERROR EXTRACT FILE HEADER: ' + str(err))
#             return None

#         try:
#             data += self.extract_optional_header(pe)
#         except Exception as err:
#             print('ERROR EXTRACT OPTIONAL HEADER: ' + str(err))
#             return None
            
#             # derived features
#             #number of suspicisou sections and non-suspicsious section
#         try:
#             num_ss_nss = self.get_count_suspicious_sections(pe)
#             data += num_ss_nss
#         except Exception as err:
#             print('ERROR get_count_suspicious_sections: ' + str(err))
#             return None

#         try:
#             packer = self.check_packer(filepath)
#             data += packer[0]
#         except Exception as err:
#             print('ERROR check packer: ' + str(err))
#             return None

#         try:
#             entropy_sections = self.get_text_data_entropy(pe)
#             data += entropy_sections
#         except Exception as err:
#             print('ERROR check entropy: ' + str(err))
#             return None

#         try:
#             f_size_entropy = self.get_file_entropy(filepath)
#             data += f_size_entropy
#         except Exception as err:
#             print('ERROR check fsize entropy: ' + str(err))
#             return None

#         try:
#             fileinfo = self.get_fileinfo(pe)
#             data.append(fileinfo)
#             data.append(self.type)
#         except Exception as err:
#             print('ERROR check fileinfo: ' + str(err))
#             return None
            
#         return data
    
#     def write_csv_data(self,data):
#         filepath = self.output
#         csv_file= open(filepath,"a")
#         writer = csv.writer(csv_file, delimiter=',')
#         writer.writerow(data)
#         csv_file.close()
    

#     def create_dataset(self):
#         self.write_csv_header()
#         #run through all file of source and extract features

#         count = 0
#         files = os.listdir(self.source)
#         lenfile = len(files)
#         for file in files:         
#                 print("Extract: {} / {}".format(count, lenfile))
#                 count += 1
#                 filepath = self.source + file
#                 data = self.extract_all(filepath)
#                 print(data)
#                 if data != None and len(data)>0:
#                     self.write_csv_data(data)
#                     print("Successfully Data extracted and written for {}.".format(file))

In [2]:
import os
import csv
import pefile
import yara
import math
from typing import List, Tuple

def extract_dos_header(pe):
    try:
        dos_header = pe.DOS_HEADER
        return [
            dos_header.e_magic,
            dos_header.e_cblp,
            dos_header.e_cp,
            dos_header.e_crlc,
            dos_header.e_cparhdr,
            dos_header.e_minalloc,
            dos_header.e_maxalloc,
            dos_header.e_ss,
            dos_header.e_sp,
            dos_header.e_csum,
            dos_header.e_ip,
            dos_header.e_cs,
            dos_header.e_lfarlc,
            dos_header.e_ovno,
            dos_header.e_oemid,
            dos_header.e_oeminfo,
            dos_header.e_lfanew
        ]
    except Exception:
        return [0] * 17

def extract_file_header(pe):
    try:
        file_header = pe.FILE_HEADER
        return [
            file_header.Machine,
            file_header.NumberOfSections,
            file_header.TimeDateStamp,
            file_header.PointerToSymbolTable,
            file_header.NumberOfSymbols,
            file_header.SizeOfOptionalHeader,
            file_header.Characteristics
        ]
    except Exception:
        return [0] * 7

def extract_optional_header(pe):
    try:
        optional_header = pe.OPTIONAL_HEADER
        return [
            optional_header.Magic,
            optional_header.MajorLinkerVersion,
            optional_header.MinorLinkerVersion,
            optional_header.SizeOfCode,
            optional_header.SizeOfInitializedData,
            optional_header.SizeOfUninitializedData,
            optional_header.AddressOfEntryPoint,
            optional_header.BaseOfCode,
            optional_header.BaseOfData,
            optional_header.ImageBase,
            optional_header.SectionAlignment,
            optional_header.FileAlignment,
            optional_header.MajorOperatingSystemVersion,
            optional_header.MinorOperatingSystemVersion,
            optional_header.MajorImageVersion,
            optional_header.MinorImageVersion,
            optional_header.MajorSubsystemVersion,
            optional_header.MinorSubsystemVersion,
            optional_header.SizeOfImage,
            optional_header.SizeOfHeaders,
            optional_header.CheckSum,
            optional_header.Subsystem,
            optional_header.DllCharacteristics,
            optional_header.SizeOfStackReserve,
            optional_header.SizeOfStackCommit,
            optional_header.SizeOfHeapReserve,
            optional_header.SizeOfHeapCommit,
            optional_header.LoaderFlags,
            optional_header.NumberOfRvaAndSizes
        ]
    except Exception:
        return [0] * 29

def get_count_suspicious_sections(pe):
    try:
        benign_sections = {b'.text', b'.data', b'.rdata', b'.idata', b'.edata', b'.rsrc', b'.bss', b'.crt', b'.tls'}
        section_names = [section.Name.strip() for section in pe.sections]
        non_suspicious = len(set(section_names).intersection(benign_sections))
        suspicious = len(section_names) - non_suspicious
        return [suspicious, non_suspicious]
    except Exception:
        return [0, 0]

def check_packer(filepath, rules):
    try:
        matches = rules.match(filepath)
        if matches:
            return [1, matches[0].rule]
        return [0, "NoPacker"]
    except Exception:
        return [0, "Error"]

def get_text_data_entropy(pe):
    try:
        text_entropy, data_entropy = 0.0, 0.0
        for section in pe.sections:
            if section.Name.startswith(b".text"):
                text_entropy = section.get_entropy()
            elif section.Name.startswith(b".data"):
                data_entropy = section.get_entropy()
        return [text_entropy, data_entropy]
    except Exception:
        return [0.0, 0.0]

def get_file_bytes_size(filepath):
    try:
        with open(filepath, "rb") as f:
            data = f.read()
        return data, len(data)
    except Exception:
        return b"", 0

def calculate_file_entropy(byte_arr, file_size):
    try:
        if file_size == 0:
            return 0.0
        freq_list = [0] * 256
        for byte in byte_arr:
            freq_list[byte] += 1
        entropy = -sum((freq / file_size) * math.log(freq / file_size, 2)
                       for freq in freq_list if freq > 0)
        return entropy
    except Exception:
        return 0.0

def extract_file_entropy(filepath):
    byte_arr, file_size = get_file_bytes_size(filepath)
    entropy = calculate_file_entropy(byte_arr, file_size)
    return [file_size, entropy]

def extract_features(pe, filepath, rules):
    features = []
    features.extend(extract_dos_header(pe))
    features.extend(extract_file_header(pe))
    features.extend(extract_optional_header(pe))
    features.extend(get_count_suspicious_sections(pe))
    features.extend(check_packer(filepath, rules))
    features.extend(get_text_data_entropy(pe))
    features.extend(extract_file_entropy(filepath))
    return features

def write_csv_header(output_path, header):
    if not os.path.exists(output_path):
        with open(output_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(header)

def write_csv_row(output_path, row):
    with open(output_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(row)

def process_pe_file(filepath, rules, output_path, label):
    try:
        pe = pefile.PE(filepath)
        features = extract_features(pe, filepath, rules)
        features.append(label)
        write_csv_row(output_path, features)
        print(f"Successfully processed {filepath}")
    except Exception as e:
        print(f"Failed to process {filepath}: {e}")

def create_dataset(source_dir, output_path, yara_rules_path, label):
    rules = yara.compile(yara_rules_path)
    header = (
        ["e_magic", "e_cblp", "e_cp", "e_crlc", "e_cparhdr", "e_minalloc", "e_maxalloc", "e_ss", "e_sp", "e_csum", "e_ip", "e_cs", "e_lfarlc", "e_ovno", "e_oemid", "e_oeminfo", "e_lfanew"] +
        ["Machine", "NumberOfSections", "TimeDateStamp", "PointerToSymbolTable", "NumberOfSymbols", "SizeOfOptionalHeader", "Characteristics"] +
        ["Magic", "MajorLinkerVersion", "MinorLinkerVersion", "SizeOfCode", "SizeOfInitializedData", "SizeOfUninitializedData", "AddressOfEntryPoint", "BaseOfCode", "BaseOfData", "ImageBase", "SectionAlignment", "FileAlignment", "MajorOperatingSystemVersion", "MinorOperatingSystemVersion", "MajorImageVersion", "MinorImageVersion", "MajorSubsystemVersion", "MinorSubsystemVersion", "SizeOfImage", "SizeOfHeaders", "CheckSum", "Subsystem", "DllCharacteristics", "SizeOfStackReserve", "SizeOfStackCommit", "SizeOfHeapReserve", "SizeOfHeapCommit", "LoaderFlags", "NumberOfRvaAndSizes"] +
        ["SuspiciousSections", "NonSuspiciousSections", "PackerDetected", "PackerType", "TextEntropy", "DataEntropy", "FileSize", "FileEntropy", "Label"]
    )
    write_csv_header(output_path, header)

    for filename in os.listdir(source_dir):
        filepath = os.path.join(source_dir, filename)
        process_pe_file(filepath, rules, output_path, label)

In [ ]:
PATH_BENIGN = './Dataset/benign/'
PATH_MALWARE = './Dataset/malware/'

# import os

# PE_EXTRACTOR = PEFeatures(source=PATH_BENIGN, output='benign.csv', label="0")
# PE_EXTRACTOR.create_dataset()

# PE_EXTRACTOR = PEFeatures(source=PATH_MALWARE, output='malware.csv', label="1")
# PE_EXTRACTOR.create_dataset()

create_dataset(source_dir=PATH_BENIGN, output_path="benign.csv", yara_rules_path="peid.yar", label="0")

Successfully processed ./Dataset/benign/DA1C13AC314555244643566A9B093AED.exe
Successfully processed ./Dataset/benign/513c8fdd0e211a18c25836f041ba0fcfb1dddf472ff91b72b6a15eb049cef184.exe
Successfully processed ./Dataset/benign/58909FF2490F23179BBB4AFD630143FB.exe
Successfully processed ./Dataset/benign/4967E31DF89F5AFEEE8DF6E3A1553D30.exe
Successfully processed ./Dataset/benign/1E2B51001B566160428896D2B19C36E4.exe
Successfully processed ./Dataset/benign/6232e33d88d2cad0dbe0fc4324f63fb2b9ba833b0d14f7fd529ab52ed7c6dacd.exe
Successfully processed ./Dataset/benign/516909EACAE216AC4F8BAC352184E560.exe
Successfully processed ./Dataset/benign/503474B8A34BD5CC4F5879BE77D81DAB.exe
Successfully processed ./Dataset/benign/2C2A06DCB3A3615DD703E9FF6F62731B.exe
Successfully processed ./Dataset/benign/db345130e2cd90ea550affa0fc43a285f09c8f15c9382d35334a1bde48eaa92e.exe
Successfully processed ./Dataset/benign/50A90B1CC20B04F8C1C36DFFA0A997CD.exe
Successfully processed ./Dataset/benign/9E72F8BCF7EAEC0A5